# ART APGD sample generation (TF2.10)

Generate APGD-CE adversarial samples using ART against:
- $\mathcal{U}^{(0)}$: baseline ensemble (4 ResNet + 4 VGG probability models)
- $\mathcal{U}^{(1)}$: first immunized generation ensemble

Outputs are saved under `data/adversarial_samples/art/` as `.npz`.

In [ ]:
from __future__ import annotations

import os
import glob
import yaml
from pathlib import Path
from typing import Dict, List, Tuple, Any, TypedDict
import numpy as np
import tensorflow as tf
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import source.custom_specialization as custom_specialization
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import AutoProjectedGradientDescent

In [ ]:
def load_yaml(path: str):
    with open(path, "r") as f:
        return yaml.safe_load(f)

PATHS = load_yaml("./configs/paths.yaml")
EXP   = load_yaml("./configs/exp.yaml")

data_root   = PATHS["data_root"]
tf_model_dir = PATHS["tf_model_dir"]
apgd_out    = PATHS["apgd_out"]

seed = int(EXP["seed"])
apgd_cfg = EXP["art_apgd"]

NORM = int(apgd_cfg["norm"])              # 2
EPS  = float(apgd_cfg["eps"])             # 0.5 (override below)
EPS_STEP = float(apgd_cfg["eps_step"])    # 0.2
MAX_ITER = int(apgd_cfg["max_iter"])      # 2
RINIT = int(apgd_cfg["nb_random_init"])   # 4
APGD_SETTINGS = [
    (EPS, EPS_STEP, MAX_ITER, RINIT),
]
APGD_OUT = Path(apgd_out)
APGD_OUT.mkdir(parents=True, exist_ok=True)

tf_model_dir = Path(tf_model_dir)

np.random.seed(seed)
tf.random.set_seed(seed)

(x_all, y_all), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

y_all  = y_all.reshape(-1).astype(np.int64)
y_test = y_test.reshape(-1).astype(np.int64)

x_train, x_val, y_train, y_val = train_test_split(
    x_all, y_all,
    test_size=0.2,
    random_state=seed,
    stratify=y_all
)

# normalize to [0,1]
x_train = x_train.astype(np.float32) / 255.0
x_val   = x_val.astype(np.float32) / 255.0
x_test  = x_test.astype(np.float32) / 255.0

y_train = y_train.astype(np.int64)
y_val   = y_val.astype(np.int64)

def make_train_chunks(x: np.ndarray, y: np.ndarray, chunk_size: int = 10000):
    out = []
    n = x.shape[0]
    for s in range(0, n, chunk_size):
        e = min(s + chunk_size, n)
        name = f"train_all_{s:04d}_{e:04d}"
        out.append((name, x[s:e], y[s:e]))
    return out

splits: List[Tuple[str, np.ndarray, np.ndarray]] = []
splits.extend(make_train_chunks(x_train, y_train, chunk_size=10000))
splits.append(("val",  x_val,  y_val))
splits.append(("test", x_test, y_test))


## Load ensembles for $\mathcal{U}^{(0)}$

We load probability-output Keras models (`.keras`) and build an average-probability ensemble for ART.

In [ ]:
U0_GLOB = [
    str(tf_model_dir / "resnet_*.keras"),
    str(tf_model_dir / "vgg*_raw.keras"),
]
U1_GLOB = [
    "./data/specialized_models/*.keras"
]

In [ ]:
def collect_model_paths(globs: List[str]) -> List[str]:
    paths = []
    for g in globs:
        paths.extend(sorted(glob.glob(g)))
    # de-dup while preserving order
    seen = set()
    out = []
    for p in paths:
        if p not in seen:
            seen.add(p)
            out.append(p)
    return out

def load_prob_models(paths: List[str]) -> List[tf.keras.Model]:
    models = []
    for p in paths:
        m = load_model(p)
        models.append(m)
    return models


class ProbAverageEnsemble(tf.keras.Model):
    def __init__(self, prob_models: List[tf.keras.Model], eps: float = 1e-12):
        super().__init__()
        self.prob_models = prob_models
        self.eps = eps

    def call(self, x, training=False):
        ps = [m(x, training=training) for m in self.prob_models]  # probs
        p = tf.add_n(ps) / float(len(ps))
        return tf.clip_by_value(p, self.eps, 1.0)
def build_art_classifier(prob_models: List[tf.keras.Model]) -> TensorFlowV2Classifier:
    ens = ProbAverageEnsemble(prob_models)
    clf = TensorFlowV2Classifier(
        model=ens,
        nb_classes=10,
        input_shape=(32, 32, 3),
        clip_values=(0.0, 1.0),
        loss_object=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    )
    return clf

In [ ]:
def save_npz(path: Path, **kwargs):
    path.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(path, **kwargs)
    print("Saved:", path.name)

In [ ]:
def run_apgdce_and_save(
    clf: TensorFlowV2Classifier,
    X: np.ndarray,
    Y: np.ndarray,
    split : str,
    out_path: Path,
    eps: float,
    eps_step: float,
    max_iter: int,
    nb_random_init: int,
    batch_size: int = 64,
    tag: str = ""
    
):
    attack = AutoProjectedGradientDescent(
        estimator=clf,
        norm=2,
        eps=float(eps),
        eps_step=float(eps_step),
        max_iter=int(max_iter),
        nb_random_init=int(nb_random_init),
        targeted=False,
        batch_size=int(batch_size),
        loss_type="cross_entropy",
        verbose=False,
    )
    x_adv = attack.generate(x=X, y=None).astype(np.float32)

    save_npz(
        out_path,
        x_adv=x_adv,
        y=Y.astype(np.int64),
        split = split,
        norm="L2",
        eps=np.float32(eps),
        eps_step=np.float32(eps_step),
        max_iter=np.int32(max_iter),
        nb_random_init=np.int32(nb_random_init),
        tag = tag
    )
    return x_adv

In [ ]:
def gen_for_ensemble(tag: str, model_paths: List[str]):
    if len(model_paths) == 0:
        print(f"[{tag}] no models found -> skipping")
        return

    prob_models = load_prob_models(model_paths)
    clf = build_art_classifier(prob_models)

    for eps, eps_step, max_iter, rinit in APGD_SETTINGS:
        eps_tag = str(eps).replace(".", "p")
        step_tag = str(eps_step).replace(".", "p")

        for split_name, Xs, Ys in splits:
            out_name = f"{split_name}_apgdce_l2_eps{eps_tag}_step{step_tag}_it{max_iter}_rinit{rinit}_against_{tag}.npz"
            out_path = APGD_OUT / out_name

            if out_path.exists():
                continue  # avoid re-generation

            _ = run_apgdce_and_save(
                clf,
                Xs.astype(np.float32),
                Ys.astype(np.int64),
                split = split_name,
                out_path=out_path,
                eps=eps,
                eps_step=eps_step,
                max_iter=max_iter,
                nb_random_init=rinit,
                batch_size=64,
                tag = tag
            )



In [ ]:
def merge_train_chunks(tag: str, eps: float, eps_step: float, max_iter: int, rinit: int):
    eps_tag = str(eps).replace(".", "p")
    step_tag = str(eps_step).replace(".", "p")

    pattern = str(APGD_OUT / f"train_all_*_apgdce_l2_eps{eps_tag}_step{step_tag}_it{max_iter}_rinit{rinit}_against_{tag}.npz")
    paths = sorted(glob.glob(pattern))
    if len(paths) == 0:
        print("No chunk files:", pattern)
        return

    x_list, y_list = [], []
    for p in paths:
        d = np.load(p)
        x_list.append(d["x_adv"])
        y_list.append(d["y"])

    x_adv = np.concatenate(x_list, axis=0).astype(np.float32)
    y = np.concatenate(y_list, axis=0).astype(np.int64)

    out_name = f"train_all_apgdce_l2_eps{eps_tag}_step{step_tag}_it{max_iter}_rinit{rinit}_against_{tag}.npz"
    out_path = APGD_OUT / out_name
    save_npz(
        out_path,
        x_adv=x_adv,
        y=y,
        split = 'train',
        norm="L2",
        eps=np.float32(eps),
        eps_step=np.float32(eps_step),
        max_iter=np.int32(max_iter),
        nb_random_init=np.int32(rinit),
        tag = tag
    )


## Generate APGD-CE samples for $\mathcal{U}^{(0)}$

We generate and save `.npz` for each split chunk to avoid memory issues.

In [ ]:
# current config is the "weak" perturbation.
u0_paths = collect_model_paths(U0_GLOB)

gen_for_ensemble("U0", u0_paths)
merge_train_chunks("U0", EPS, EPS_STEP, MAX_ITER, RINIT)


# Get $\mathcal{U}^{(1)}$

In [ ]:
SPECIAL_DIR = Path("./data/specialized_models")


In [ ]:
TrainData = Tuple[np.ndarray, np.ndarray]
ValData = Tuple[np.ndarray, np.ndarray]

class HistoryDict(TypedDict, total=False):
    history: Dict[str, List[float]]
    params: Dict[str, Any]
    epoch: List[int]


class SpecializeResult(TypedDict):
    model_path: str   
    history: HistoryDict  

        
def specialize(
    new_train : TrainData,
    new_val : ValData,
    original_model : tf.keras.Model,
    new_model_path : str = 'specialized_model.keras',
    path : Path = Path('./data/specialized_models/'),
    verbose = 1,
    name : str = ''
) -> dict:
    """
    Returns (baseline_adv_model, tuned_baseline_adv_model, tuned_history_dict_or_None)
    """
    
    
    x_tr, y_tr = new_train
    x_v, y_v = new_val
    if y_tr.ndim == 1 or y_tr.shape[1] != 10:
        y_tr = to_categorical(y_tr, 10)
    if y_v.ndim == 1 or y_v.shape[1] != 10:
        y_v = to_categorical(y_v, 10)
    if not new_model_path.lower().endswith('.keras'):
        new_model_path += '.keras'
        
    model_path  = path /  Path(new_model_path)    

    if model_path.exists():
        specialized_model = load_model(model_path)
        print(f'{model_path} already exists.')
        hist = custom_specialization.load_history(model_path) 
        
        if hist is None:
            print(f"[WARN] No history found for {model_path}. History is empty dictionary.")
            hist = {}
    else:
        print(f'{model_path} training...')
        specialized_model,hist = custom_specialization.turn_specialist(original_model, path = model_path,
                                                x_tr=x_tr, y_tr=y_tr,
                                                  x_v=x_v,   y_v=y_v,
                                                  epochs=21, learning_rate=1e-3, batch_size=128, verbose=verbose, name=f"tuned_once{name}")
        hist = {"history": hist.history, "params": hist.params, "epoch": hist.epoch}
        specialized_model.save(model_path)
    return {
    "model_path": str(model_path),
    "history": hist,
}, specialized_model

In [ ]:
samples = {}
y_true_dict = {}
for p in os.listdir(APGD_OUT):
    d = np.load(APGD_OUT/Path(p))
    adv_name = 'APGD_weak' if d['max_iter'] == 2 else 'APGD_strong'
    split = str(d['split'])
    sample_name = adv_name + '_' + split
    samples[sample_name] = d['x_adv']
    y_true_dict[sample_name] = d['y']

x_temp = np.concatenate([samples['APGD_weak_train'],samples['APGD_weak_val']],axis=0)
y_temp = np.concatenate([y_true_dict['APGD_weak_train'],y_true_dict['APGD_weak_val']],axis=0)

new_stratified_x_train, new_stratified_x_val, new_stratified_y_train, new_stratified_y_val = train_test_split(
        x_temp,
        y_temp,
        test_size=0.2,
        random_state=42,
        stratify=y_temp
    )

specialized_model_dict={}

model_dict = {}
for f_name in sorted(os.listdir("./data/models/")):
    if 'original' not in f_name and f_name.endswith('keras'):
        print(f_name)
        m = load_model(f"./data/models/{f_name}")
        model_dict[f_name] = m
adv_sample_name = 'APGD_weak'
for model_name, m in model_dict.items():
    model_name = model_name[:-6]
    model_name = model_name.split('_')[0] if model_name[0] == 'v' else model_name.split('_')[0] + model_name.split('_')[-1] + 'v'+model_name.split('_')[1][-1]
    info = specialize(
        (new_stratified_x_train, to_categorical(new_stratified_y_train,10)),
        (new_stratified_x_val, to_categorical(new_stratified_y_val,10)),
        m,
        new_model_path=f"{model_name}_{adv_sample_name}-SP")
model_dict = {}
for f_name in sorted(os.listdir("./data/models/")):
    if 'original' not in f_name and f_name.endswith('keras'):
        print(f_name)
        m = load_model(f"./data/models/{f_name}")
        model_dict[f_name] = m
        
SPs_GLOB = [
    "./data/specialized_models/*.keras"
]
U1_GLOB = U0_GLOB + SPs_GLOB


In [ ]:
SPs_paths = collect_model_paths(SPs_GLOB)

gen_for_ensemble("SPs", SPs_paths)
merge_train_chunks("SPs", EPS, EPS_STEP, MAX_ITER, RINIT)

u1_paths = collect_model_paths(U1_GLOB)

gen_for_ensemble("U1", u1_paths)
merge_train_chunks("U1", EPS, EPS_STEP, MAX_ITER, RINIT)

In [ ]:
# strong configuration.
EPS, EPS_STEP, MAX_ITER, RINIT = (0.7, 0.2, 10, 4)


gen_for_ensemble("U0", u0_paths)
merge_train_chunks("U0", EPS, EPS_STEP, MAX_ITER, RINIT)


gen_for_ensemble("SPs", SPs_paths)
merge_train_chunks("SPs", EPS, EPS_STEP, MAX_ITER, RINIT)


gen_for_ensemble("U1", u1_paths)
merge_train_chunks("U1", EPS, EPS_STEP, MAX_ITER, RINIT)

